In [1]:
import overpass
api = overpass.API()

In [2]:
latitude = 55.8962
longitude = 37.3992

In [3]:
raw_data = api.get(
    f'way[landuse="residential"]({latitude - .001},{longitude - .001},{latitude + .001},{longitude + .001});',
    verbosity='geom'
)

In [6]:
features = raw_data['features']

In [10]:
features

[{"geometry": {"coordinates": [[37.397803, 55.896186], [37.398473, 55.89495], [37.398941, 55.894039], [37.398989, 55.893996], [37.399072, 55.893971], [37.399174, 55.893939], [37.400376, 55.893996], [37.401633, 55.894037], [37.401738, 55.894056], [37.40181, 55.894096], [37.40184, 55.894141], [37.40184, 55.894194], [37.401806, 55.894263], [37.401785, 55.894293], [37.401632, 55.894524], [37.400503, 55.896382], [37.400463, 55.896457], [37.399484, 55.898106], [37.399456, 55.898133], [37.399424, 55.898153], [37.399367, 55.898174], [37.399299, 55.898188], [37.399268, 55.898188], [37.39921, 55.898189], [37.396982, 55.898146], [37.396934, 55.898141], [37.396878, 55.898122], [37.396839, 55.898096], [37.396819, 55.898068], [37.39682, 55.898038], [37.397803, 55.896186]], "type": "LineString"}, "id": 134225763, "properties": {"landuse": "residential", "residential": "urban"}, "type": "Feature"}]

In [35]:
max_lat, min_lat, max_lon, min_lon = 0, 0, 0, 0
for pair in features[0]['geometry']['coordinates']:
    lat, lon = pair
    max_lat = max(lat, max_lat or lat)
    max_lon = max(lon, max_lon or lon)        
    min_lat = min(lat, min_lat or lat)
    min_lon = min(lon, min_lon or lon)

[37.397803, 55.896186]
[37.398473, 55.89495]
[37.398941, 55.894039]
[37.398989, 55.893996]
[37.399072, 55.893971]
[37.399174, 55.893939]
[37.400376, 55.893996]
[37.401633, 55.894037]
[37.401738, 55.894056]
[37.40181, 55.894096]
[37.40184, 55.894141]
[37.40184, 55.894194]
[37.401806, 55.894263]
[37.401785, 55.894293]
[37.401632, 55.894524]
[37.400503, 55.896382]
[37.400463, 55.896457]
[37.399484, 55.898106]
[37.399456, 55.898133]
[37.399424, 55.898153]
[37.399367, 55.898174]
[37.399299, 55.898188]
[37.399268, 55.898188]
[37.39921, 55.898189]
[37.396982, 55.898146]
[37.396934, 55.898141]
[37.396878, 55.898122]
[37.396839, 55.898096]
[37.396819, 55.898068]
[37.39682, 55.898038]
[37.397803, 55.896186]


In [36]:
 max_lon, max_lat,  min_lon, min_lat,

(55.898189, 37.40184, 55.893939, 37.396819)

In [47]:
from shapely.geometry import Polygon
from pyproj import Proj, transform, Transformer
from geopy.distance import geodesic

In [53]:
def geodesic_area(coordinates):
    if len(coordinates) < 3:
        return 0 

    tranformer = Transformer.from_crs("EPSG:4326", 'EPSG:3857')
    
    # Project the coordinates to UTM
    utm_coords = [tranformer.transform(lon, lat) for lat, lon in coordinates]

    # Create a polygon with shapely
    poly = Polygon(utm_coords)

    # Calculate the area in square meters
    area = poly.area
    return area

In [54]:
geodesic_area(features[0]['geometry']['coordinates'])

260571.66842714194

In [55]:
poly = Polygon(features[0]['geometry']['coordinates'])


In [68]:
min_lat,min_lon,max_lat,max_lon = poly.bounds

In [101]:
raw_buildings_data = api.get(
    f'way[building=apartments]({min_lon},{min_lat},{max_lon},{max_lat});',
    verbosity='geom'
)

In [102]:
raw_buildings_data

{"features": [{"geometry": {"coordinates": [[37.395691, 55.89531], [37.395836, 55.895299], [37.396031, 55.895184], [37.3964, 55.895105], [37.396362, 55.895037], [37.396783, 55.894955], [37.396813, 55.895016], [37.397044, 55.895056], [37.397159, 55.894997], [37.397126, 55.894933], [37.397016, 55.89488], [37.396781, 55.894896], [37.396747, 55.894832], [37.396265, 55.894925], [37.396302, 55.894993], [37.395859, 55.895076], [37.395652, 55.895184], [37.395619, 55.895265], [37.395691, 55.89531]], "type": "LineString"}, "id": 37824476, "properties": {"addr:city": "Москва", "addr:country": "RU", "addr:housenumber": "9 к1", "addr:postcode": "125466", "addr:street": "улица Соловьиная Роща", "building": "apartments", "building:flats": "94", "building:levels": "7", "start_date": "2003"}, "type": "Feature"}, {"geometry": {"coordinates": [[37.397599, 55.895088], [37.3978, 55.895121], [37.397472, 55.895691], [37.397375, 55.89582], [37.397264, 55.895856], [37.397016, 55.895841], [37.396492, 55.895835]

In [103]:
buildings_area = 0
for feature in raw_buildings_data['features']:
    area = geodesic_area(feature['geometry']['coordinates'])
    buildings_area += area

In [104]:
print(buildings_area)

53125.27399269093
